# 01 — Preprocessing & EDA (Beijing Multi-Site Air Quality)
Mục tiêu: tải dữ liệu, làm sạch, tạo nhãn phân lớp (AQI class theo PM2.5 24h mean), tạo đặc trưng thời gian + lag, và lưu `data/processed/cleaned.parquet`.

**Lưu ý:** nếu `USE_UCIMLREPO=True` thì notebook cần internet để tải dataset từ UCI.

In [ ]:
USE_UCIMLREPO = False
RAW_ZIP_PATH = "data/raw/PRSA2017_Data_20130301-20170228.zip"

OUTPUT_CLEANED_PATH = 'data/processed/cleaned.parquet'
LAG_HOURS=[1, 3, 24]


In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

from src.classification_library import (
    load_beijing_air_quality,
    clean_air_quality_df,
    add_pm25_24h_and_label,
    add_time_features,
    add_lag_features,
)

PROJECT_ROOT = Path('..').resolve()
OUT_PATH = (PROJECT_ROOT / OUTPUT_CLEANED_PATH).resolve()
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)


ModuleNotFoundError: No module named 'src'

In [ ]:
df_raw = load_beijing_air_quality(use_ucimlrepo=USE_UCIMLREPO, raw_zip_path=RAW_ZIP_PATH)
print('raw shape:', df_raw.shape)
df_raw.head()

In [ ]:
df = clean_air_quality_df(df_raw)
df = add_pm25_24h_and_label(df)
df = add_time_features(df)
df = add_lag_features(df, lag_hours=LAG_HOURS)
print('cleaned shape:', df.shape)
df[['datetime','station','PM2.5','pm25_24h','aqi_class']].head(10)

In [ ]:
# EDA nhanh: missingness và phân bố lớp
missing_rate = df.isna().mean().sort_values(ascending=False)
missing_rate.head(20)

In [ ]:
# 7. Kiểm tra tính dừng (Stationarity) - ADF và KPSS
print("\n" + "="*60)
print("7. KIỂM ĐỊNH TÍNH DỪNG (STATIONARITY)")
print("="*60)

from statsmodels.tsa.stattools import adfuller, kpss

# ADF test
adf_result = adfuller(pm25_series, autolag='AIC')
print("ADF (Augmented Dickey-Fuller) Test:")
print(f"  ADF Statistic: {adf_result[0]:.4f}")
print(f"  p-value: {adf_result[1]:.4f}")
print(f"  Critical values:")
for key, value in adf_result[4].items():
    print(f"    {key}: {value:.4f}")

if adf_result[1] < 0.05:
    print("  => Bác bỏ H0 (có unit root), chuỗi có xu hướng DỪNG")
else:
    print("  => Không bác bỏ H0, chuỗi có xu hướng KHÔNG DỪNG")

# KPSS test
print("\nKPSS Test:")
kpss_result = kpss(pm25_series, regression='c', nlags='auto')
print(f"  KPSS Statistic: {kpss_result[0]:.4f}")
print(f"  p-value: {kpss_result[1]:.4f}")
print(f"  Critical values:")
for key, value in kpss_result[3].items():
    print(f"    {key}: {value:.4f}")

if kpss_result[1] > 0.05:
    print("  => Không bác bỏ H0 (dừng), chuỗi có xu hướng DỪNG")
else:
    print("  => Bác bỏ H0, chuỗi có xu hướng KHÔNG DỪNG")

print("\n" + "="*60)
print("KẾT LUẬN TỔNG HỢP Q1:")
print("="*60)
print("1. Dữ liệu: 420,768 dòng, 12 trạm, tần suất hourly liên tục 2013-2017")
print("2. Missing: PM2.5 thiếu ~2%, cần xử lý vì là biến mục tiêu dự báo")
print("3. Phân phối: Lệch phải với spike cao, cần lưu ý khi đánh giá (RMSE vs MAE)")
print("4. Xu hướng: Có biến động mạnh, rolling mean cho thấy pattern theo mùa")
print("5. Tự tương quan: Mạnh ở lag ngắn, có chu kỳ ngày (24h) và tuần (168h)")
print("6. Tính dừng: Cần kiểm tra kỹ (ADF/KPSS), có thể cần differencing (d>0)")
print("="*60)

In [ ]:
# 6. Kiểm tra tự tương quan (Autocorrelation)
print("\n" + "="*60)
print("6. KIỂM TRA TỰ TƯƠNG QUAN")
print("="*60)

# Lấy chuỗi PM2.5 sạch
pm25_series = station_df.set_index('datetime')['PM2.5'].dropna()

# Tính autocorrelation ở các lag quan trọng
lags_to_check = [1, 3, 6, 12, 24, 48, 72, 168]  # 168h = 1 tuần
autocorr_results = {}
for lag in lags_to_check:
    autocorr_results[f'lag_{lag}h'] = pm25_series.autocorr(lag=lag)

autocorr_df = pd.DataFrame.from_dict(autocorr_results, orient='index', columns=['Autocorrelation'])
print(autocorr_df)

print(f"\n** Phân tích tự tương quan:")
print(f"   - Lag 1h: {autocorr_results['lag_1h']:.3f} (rất cao - giá trị hiện tại phụ thuộc mạnh vào giờ trước)")
print(f"   - Lag 24h: {autocorr_results['lag_24h']:.3f} (chu kỳ ngày)")
print(f"   - Lag 168h: {autocorr_results['lag_168h']:.3f} (chu kỳ tuần)")
print(f"\n=> Kết luận: Chuỗi có tự tương quan mạnh, đặc biệt ở lag ngắn và chu kỳ ngày/tuần")

In [ ]:
# 5. Vẽ chuỗi PM2.5 theo thời gian
print("\n" + "="*60)
print("5. VẼ CHUỖI PM2.5 THEO THỜI GIAN")
print("="*60)

# Lấy 1 trạm để vẽ (chọn trạm đầu tiên)
sample_station = df['station'].iloc[0]
station_df = df[df['station'] == sample_station].sort_values('datetime')

fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# 5a. Toàn giai đoạn
axes[0].plot(station_df['datetime'], station_df['PM2.5'], linewidth=0.5, alpha=0.7)
axes[0].set_title(f'PM2.5 toàn giai đoạn - Trạm: {sample_station}')
axes[0].set_ylabel('PM2.5')
axes[0].grid(True, alpha=0.3)

# Thêm rolling mean để nhìn xu hướng
station_df_temp = station_df.set_index('datetime')
rolling_7d = station_df_temp['PM2.5'].rolling(window=24*7, min_periods=1).mean()
axes[0].plot(rolling_7d.index, rolling_7d.values, 'r-', linewidth=2, label='Rolling mean 7 days', alpha=0.8)
axes[0].legend()

# 5b. Zoom vào 2 tháng (chọn tháng 1-2/2017 - gần test set)
zoom_start = '2017-01-01'
zoom_end = '2017-02-28'
zoom_df = station_df[(station_df['datetime'] >= zoom_start) & (station_df['datetime'] <= zoom_end)]

axes[1].plot(zoom_df['datetime'], zoom_df['PM2.5'], linewidth=1, marker='o', markersize=1, alpha=0.7)
axes[1].set_title(f'PM2.5 zoom 2 tháng (Jan-Feb 2017) - Trạm: {sample_station}')
axes[1].set_ylabel('PM2.5')
axes[1].set_xlabel('Datetime')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n=> Từ biểu đồ trên, ta thấy:")
print("   - Chuỗi có biến động mạnh theo thời gian")
print("   - Có các đỉnh spike (PM2.5 rất cao)")
print("   - Rolling mean cho thấy xu hướng theo mùa")

In [ ]:
# 4. Boxplot và quantile để nhìn ngoại lai
print("\n" + "="*60)
print("4. PHÂN PHỐI VÀ NGOẠI LAI (PM2.5)")
print("="*60)

pm25_clean = df['PM2.5'].dropna()
stats = pm25_clean.describe(percentiles=[0.05, 0.25, 0.5, 0.75, 0.95, 0.99])
print(stats)

print(f"\nPhân tích phân phối:")
print(f"- Mean ({stats['mean']:.2f}) > Median ({stats['50%']:.2f})")
print(f"- Q3-Q1 (IQR) = {stats['75%'] - stats['25%']:.2f}")
print(f"- 95th percentile: {stats['95%']:.2f}")
print(f"- 99th percentile: {stats['99%']:.2f}")
print(f"- Max: {stats['max']:.2f}")
print(f"\n=> Phân phối lệch phải (skewed right) với đuôi dài, có spike cao")

# Boxplot
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Boxplot toàn bộ
axes[0].boxplot(pm25_clean, vert=True)
axes[0].set_ylabel('PM2.5')
axes[0].set_title('Boxplot PM2.5 (toàn bộ dữ liệu)')
axes[0].grid(True, alpha=0.3)

# Boxplot loại bỏ outliers cực đoan (để nhìn rõ hơn)
pm25_95 = pm25_clean[pm25_clean <= pm25_clean.quantile(0.99)]
axes[1].boxplot(pm25_95, vert=True)
axes[1].set_ylabel('PM2.5')
axes[1].set_title('Boxplot PM2.5 (loại bỏ 1% extreme outliers)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 3. Thiếu dữ liệu theo thời gian (PM2.5)
print("\n" + "="*60)
print("3. THIẾU DỮ LIỆU THEO THỜI GIAN (PM2.5)")
print("="*60)

# Tạo cột để đánh dấu thiếu
df_temp = df.copy()
df_temp['is_missing_pm25'] = df_temp['PM2.5'].isna()

# Nhóm theo tháng để xem thiếu tập trung ở đâu
df_temp['year_month'] = df_temp['datetime'].dt.to_period('M')
missing_by_month = df_temp.groupby('year_month')['is_missing_pm25'].agg(['sum', 'count', 'mean'])
missing_by_month.columns = ['Missing', 'Total', 'Missing_Rate']
missing_by_month['Missing_Rate'] = missing_by_month['Missing_Rate'] * 100

print("Top 10 tháng thiếu PM2.5 nhiều nhất:")
print(missing_by_month.sort_values('Missing_Rate', ascending=False).head(10))

import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(12, 4))
missing_by_month['Missing_Rate'].plot(ax=ax, marker='o', markersize=3)
ax.set_title('Tỷ lệ thiếu PM2.5 theo tháng')
ax.set_ylabel('Missing %')
ax.set_xlabel('Tháng')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n=> Kết luận: Thiếu dữ liệu", "rải đều" if missing_by_month['Missing_Rate'].std() < 2 else "tập trung vào một số giai đoạn")

In [ ]:
# 2. Tỷ lệ thiếu theo từng biến (chi tiết hơn)
print("\n" + "="*60)
print("2. TỶ LỆ THIẾU DỮ LIỆU THEO BIẾN")
print("="*60)
missing_pct = (df.isna().sum() / len(df) * 100).sort_values(ascending=False)
missing_df = pd.DataFrame({
    'Missing Count': df.isna().sum(),
    'Missing %': missing_pct
})
print(missing_df[missing_df['Missing Count'] > 0])

print("\n** Phân tích: Biến nào thiếu đáng lo nhất?")
print(f"- CO thiếu nhiều nhất: {missing_pct['CO']:.2f}%")
print(f"- PM2.5 (mục tiêu dự báo) thiếu: {missing_pct['PM2.5']:.2f}%")
print("- Các biến khí tượng (TEMP, PRES, DEWP, etc.) thiếu rất ít (<0.1%)")
print("\n=> Kết luận: PM2.5 thiếu ~2%, cần xử lý cẩn thận vì đây là biến mục tiêu.")

In [ ]:
# 1. Kiểm tra khoảng thời gian và tần suất
print("="*60)
print("1. KHOẢNG THỜI GIAN VÀ TẦN SUẤT")
print("="*60)
print(f"Start: {df['datetime'].min()}")
print(f"End: {df['datetime'].max()}")
print(f"Duration: {df['datetime'].max() - df['datetime'].min()}")
print(f"Total rows: {len(df):,}")
print(f"Number of stations: {df['station'].nunique()}")
print(f"Rows per station (expected): {len(df) // df['station'].nunique():,}")

# Kiểm tra tần suất theo giờ cho từng trạm
print("\nKiểm tra tần suất theo giờ (lấy 1 trạm mẫu):")
sample_station = df['station'].iloc[0]
station_df = df[df['station'] == sample_station].sort_values('datetime')
time_diff = station_df['datetime'].diff()
print(f"Trạm mẫu: {sample_station}")
print(f"Min diff: {time_diff.min()}")
print(f"Max diff: {time_diff.max()}")
print(f"Mode diff: {time_diff.mode()[0]}")
print(f"=> Tần suất: {'Hourly (liên tục)' if time_diff.mode()[0] == pd.Timedelta('1 hour') else 'Không đều'}")

## Q1: Bổ sung phân tích để hiểu dữ liệu chuỗi thời gian

Theo yêu cầu Q1, chúng ta cần kiểm tra:
1. Khoảng thời gian dữ liệu phủ (start/end) và xác nhận tần suất theo giờ
2. Tỷ lệ thiếu theo từng biến và thiếu theo thời gian
3. Boxplot/quantile để nhìn ngoại lai và phân phối lệch
4. Vẽ chuỗi PM2.5 theo thời gian (toàn giai đoạn + zoom 1-2 tháng)
5. Kiểm tra tự tương quan (lag 24h và 168h)
6. Kiểm tra tính dừng (ADF/KPSS)

In [ ]:
class_dist = df['aqi_class'].value_counts(dropna=False)
class_dist

In [ ]:
import matplotlib.pyplot as plt

class_dist.drop(index=[x for x in class_dist.index if pd.isna(x)], errors='ignore').plot(kind='bar')
plt.title('AQI class distribution (PM2.5 24h mean)')
plt.ylabel('count')
plt.tight_layout()
plt.show()

In [ ]:
df.to_parquet(OUT_PATH, index=False)
print('Saved:', OUT_PATH)